### Imports


Install the following packages:

```bash
!pip install -q vertexai
!pip install -q google-cloud-pipeline-components
!pip install -q kfp
```


In [ ]:
from kfp.dsl import pipeline
from kfp.dsl import component
from kfp import compiler

from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs

### Loading Environment Variables


In [ ]:
from dotenv import dotenv_values

config = dotenv_values(".env")

PROJECT_ID = config["PROJECT_ID"]
PIPELINE_ROOT = config["PIPELINE_ROOT"]
LOCATION = config["LOCATION"]

### Initializing AI Platform


In [ ]:
aiplatform.init(
    project=PROJECT_ID,
    staging_bucket=PIPELINE_ROOT,
    location=LOCATION,
)

This connects to Vertex AI in Google Cloud.


### Defining Pipeline Components


In [ ]:
@component
def add(a: int, b: int) -> int:
    return a + b

If you get a warning about Python 3.7, you can ignore it.


In [ ]:
@component()
def multiply_2(a: int) -> int:
    return a * 2

Components are the building blocks of Kubeflow Pipelines. Each component is a self-contained set of code that performs one step in the ML workflow.


### Defining Pipeline


In [ ]:
@pipeline
def simple_pipeline(a: int = 1, b: int = 2):
    add_task = add(a=a, b=b)
    multiply_2_task = multiply_2(a=add_task.output)

The pipeline automatically creates a Kubeflow pipeline.

The pipeline is DAG (Directed Acyclic Graph) of tasks. Each task is a component that performs one step in the pipeline.


### Compiling Pipeline


In [ ]:
compiler.Compiler().compile(
    pipeline_func=simple_pipeline, package_path="pipelines/simple_pipeline.json"
)

In [ ]:
!cat pipelines/simple_pipeline.json

This JSON file is the compiled pipeline.

It is a serialized representation of the pipeline that can be run using the Kubeflow Pipelines API.

> Note: The JSON file contain everything needed to run the pipeline, including the any secret values. Make sure to remove any sensitive information before sharing it.


### Compiling Components


In [ ]:
compiler.Compiler().compile(multiply_2, "components/multiply_2.yaml")

In [ ]:
!cat components/multiply_2.yaml

You can compile components individually as well. This is useful when you want to share components with others.


### Running Pipeline


In [ ]:
pipeline_job = pipeline_jobs.PipelineJob(
    display_name="simple-pipeline",
    template_path="pipelines/simple_pipeline.json",
    parameter_values={"a": 43, "b": 24},
)
pipeline_job.run()

`job.run(sync=True)` will run the pipeline synchronously, meaning the cell will not finish executing until the pipeline is complete.

If you want to run the pipeline asynchronously, you can use `job.run()`.


### Python Container Component

by using the `@component` decorator, you can create a component from a Python function.
Giving `base_image` argument to the decorator, you can specify the base image for the container.
This makes it a python-containerized component.

For example, the following code creates a Python container component from a Python function:

```python
@component(base_image='python:3.9')
def my_component():
    ...
```


### Scheduling Pipelines


In [ ]:
pipeline_job_schedule = aiplatform.PipelineJobSchedule(
    pipeline_job=pipeline_job,
    display_name="test_schedule",
)

pipeline_job_schedule.create(
    cron="0 * * * *",  # Run every hour
)

---


Pipeline API Documentation: https://google-cloud-pipeline-components.readthedocs.io/en/latest/


Guides from Google:

- https://cloud.google.com/vertex-ai/docs/pipelines/build-pipeline
- https://codelabs.developers.google.com/vertex-pipelines-intro#0


Sample Notebooks from Google: https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/main/notebooks/official/pipelines


Youtube Video Tutorial on Piplines:

- https://www.youtube.com/watch?v=ayv0-rC8W1Q&
- https://www.youtube.com/watch?v=ACkRqm7DSZQ
